# Deep Deterministic Policy Gradient
### This notebook contains the code for creating a DDPG agent for solving the environment Reacher

#### Logic

Following this repository https://github.com/enginBozkurt/Deep-Reinforcement-Learning-for-Enterprise-Nanodegree/tree/master/Project%202%20-Continuous%20Control and some bits of my DQN Pong code \
Idea is follows:
1) Create actor, actor target, critic & critic target models \
2) For every episode, pass state to actor to predict action \
3) Pass action to env.step() to obtain next_state, reward & terminate \
4) Continuously add them to memory until a certain length is achieved \
5) After which, we also concurrently do weights updates by randomly selecting batch_size of samples from agent's memory for model training

When mean reward of last 100 episodes is less than threshold, save model weights

#### Import modules

Please download models.py & memory.py into same directory before importing

In [1]:
import gym
import copy
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
from models import *
from memory import *

We define our hyprparamters here:
1) gamma is the discount factor multiplied to value of next state. We use this in th equation: q_target = reward + (1-done) * q_next * gamma to obtain our target action values for model training \
2) tau is used to do a gradual update of our target model weights. We use this in the equation: target_weights = (1-tau) * target_weights + tau * model_weights
3) num_states refer to the number of state variables. More details can be found here[https://www.gymlibrary.ml/environments/mujoco/reacher/] \
4) num_output refers to output size for critic model. We output an action value for every pair of actions + num_states \
5) min_memory_len is the minimum amount of historical experiences required before model training is allowed \
6) max_memory_size refers to maximum length of buffer deque. A deque holds only the latest max_memory_size of experiences with the older oones discarded. Here, we set it equal to min_memory_len \
7) epsilon_start is initial value for epsilon. For every iteration, we decay epsilon using the equation epsilon = max(episilon_min,epsilon_cay * epsilon). This is done to encourage gradual decrease of exploratory actions \
8) We set epsilon_min to 0.03 to ensure a minute chance of taking exploratory action towards the end of training \
9) We also define learning rates for both actor & critic models \
10) Finally, set device to GPU to enhance training speed. Else, use CPU

In [2]:
# episodes = 10000
gamma = 0.99
tau = 0.01
num_states = 11
num_output = 1
hidden_size = 256
batch_size = 128
num_actions = 2
min_memory_len = max_memory_size = 50000
epsilon_start = 1.0
epsilon_decay = 0.99999
epsilon_min = 0.03
critic_learning_rate = 0.001
actor_learning_rate = 0.0001
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Define classes 

We create class DDPG to define our agent behavior: \
&nbsp; a) __init__() initialises parameters used for agent-training. It starts by assigning input env to env variable, creates an experience buffer called buffer, resets &emsp; total_reward to 0 & calls _reset() method \
&nbsp; b) _reset() resets env & total_reward \
&nbsp; c) select_optimal_action() is used to observe agent's behavior after training. This will enable agent to take the action that maximises it's action-value at every &emsp; state \
&nbsp; d) select_action() chooses action based on action probability. If it is < epsilon, we take a random action. Else, act greedy \
&nbsp; e) get_experience() adds replay experiences to agent memory. Logic is as follows: \
&emsp; 1) Create episode_reward variable & set as None \
&emsp; 2) Select next action by passing actor_model, epsilon & device to select_action() \
&emsp; 3) Next action is passed to env.step(), to obtain next_state, reward, terminate & info \
&emsp; 4) Create Experience namedtuple using self.state,action,reward,terminate,next_state \
&emsp; 5) Append Experience to experience buffer \
&emsp; 6) Set state as next_state, increment total_reward \
&emsp; 7) IF next state is terminal, set episode_reward as total_reward. Print episode_reward. Call _reset() method, return True with \
&emsp; episode_reward \
&emsp; 8) ELSE, return False & episode_reward[with None value] \
&emsp; 9) IF we have collected enough memory, update_weights() & update_target_weights() \
&nbsp; f) update_target_weights() implements gradual update of target model weights with their current models using the equation target_weights = (1-tau) * \
&emsp; target_weights + tau * model_weights. Since this is a 'soft' update, we can implement it concurrently with update_weights() \
&nbsp; g) update_weights() is used to update model weights. Logic is as follows: \
&emsp; 1) Call sample method of experience buffer. Assign result as batch variable \
&emsp; 2) Assign resulting arrays to states, actions, rewards, dones, next_states\
&emsp; 3) Create tensors states_t,next_states_t, actions_t, rewards_t & done_mask \
&emsp; 4) Pass states_t, actions_t to critic_model to obtain action value prediction[q_values] \
&emsp; 5) Pass next_states_t to actor_target to obtain next state action[next_actions_t] \
&emsp; 6) Pass next_states_t, next_actions_t to critic_target to obtain next state action[q_next] \
&emsp; 7) Calculate q_targets using q_next * gamma * (1-done_mask) + rewards_t \
&emsp; 8) Calculate critic_loss using nn.MSELoss()(q_values, q_targets) & do backward propagation \
&emsp; 9) Calculate actor_loss using -critic_model(states_t, actor_model(states_t)).mean() & do backward propagation \
&emsp; 10) Update optimizers \
&emsp; 11) Update weights of target models

In [3]:
class DDPG:
    def __init__(self, env, buffer):
        self.env = env
        self.buffer = buffer
        self._reset()
        
    def _reset(self):
        self.state = env.reset()
        self.total_reward = 0
        
    def select_optimal_action(self, actor_model, device):
        state = torch.tensor(self.state).float().unsqueeze(0).to(device)
        action = actor_model(state).cpu().detach().numpy()[0]
        return action
        
    def select_action(self, actor_model, epsilon, device):
        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            state = torch.tensor(self.state).float().unsqueeze(0).to(device)
            action = actor_model(state).cpu().detach().numpy()[0]
        return action
    
    def get_experience(self, actor_model, actor_target, critic_model, critic_target, epsilon, device):
        episode_reward = None
        action = self.select_action(actor_model, epsilon, device)
        next_state, reward, terminate, info = self.env.step(action)
        exp = Experience(self.state,action,reward,terminate,next_state)
        self.buffer.append(exp)
        self.state = next_state
        self.total_reward += reward
        
        if terminate:
            episode_reward = self.total_reward
            print(f"Score {episode_reward}")
            self._reset()
            return True, episode_reward
        
        if len(buffer) == min_memory_len:
            self.update_weights(actor_model,actor_target,critic_model,critic_target) 
            
        return False, episode_reward    
        
    def update_target_weights(self, actor_model, actor_target, critic_model, critic_target):
        for target_param, param in zip(actor_target.parameters(), actor_model.parameters()):
            target_param.data.copy_(param.data * tau + target_param.data * (1.0 - tau))
       
        for target_param, param in zip(critic_target.parameters(), critic_model.parameters()):
            target_param.data.copy_(param.data * tau + target_param.data * (1.0 - tau))
        
    def update_weights(self, actor_model, actor_target, critic_model, critic_target):
        batch = buffer.sample(batch_size)
        states, actions, rewards, dones, next_states = batch
        
        states_t = torch.tensor(states).to(device)
        next_states_t = torch.tensor(next_states).to(device)
        actions_t = torch.tensor(actions).to(device)
        rewards_t = torch.tensor(rewards).to(device)
        done_mask = torch.tensor(dones).to(device)
        
        q_values = critic_model(states_t,actions_t)
        next_actions = actor_target(next_states_t)
        q_next = critic_target(next_states_t,next_actions.detach())
        q_next = q_next.detach()
        q_targets = q_next*gamma*(1-done_mask.unsqueeze(1)) + rewards_t.unsqueeze(1)
        
        critic_loss = nn.MSELoss()(q_values, q_targets)
        actor_loss = -critic_model(states_t, actor_model(states_t)).mean()
        
        actor_optimizer.zero_grad()
        actor_loss.backward()
        actor_optimizer.step()

        critic_optimizer.zero_grad()
        critic_loss.backward() 
        critic_optimizer.step()
  
        self.update_target_weights(actor_model, actor_target, critic_model, critic_target)
#         print(f"learns {self.learns}: target models weights updated")

#### Train agent

We start by creating env using gym.make() \
Next, we create our actor, actor_target, critic & critic_target models with their respective optimizers \
Then, we create experience buffer using ExperienceReplay()  & agent using DDPG() \
Finally, set epsilon to epislon_start & create empty list episode_rewards to contain each episode's rewards 

In [4]:
env = gym.make('Reacher-v4')

actor = Actor(num_states, hidden_size, num_actions).to(device)
actor_target = copy.deepcopy(actor).to(device)
actor_optimizer = optim.Adam(actor.parameters(), lr=actor_learning_rate)
critic = Critic(num_states + num_actions, hidden_size, num_output).to(device)
critic_target = copy.deepcopy(critic).to(device)
critic_optimizer = optim.Adam(critic.parameters(), lr=critic_learning_rate)

buffer = ExperienceReplay(max_memory_size)
agent = DDPG(env, buffer)
epsilon = epsilon_start
episode_rewards = []

C:\ProgramData\Anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:69: UserWarning: WARN: Agent's minimum action space value is -infinity. This is probably too low.
  logger.warn(
C:\ProgramData\Anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:73: UserWarning: WARN: Agent's maximum action space value is infinity. This is probably too high
  logger.warn(
C:\ProgramData\Anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


Let's begin agent training \
While our mean reward for previous 100 episodes is < -6, we shall do the following:
1) Set terminate to False \
2) While NOT terminate: \
&nbsp; a) Set epsilon as max(epsilon*epsilon_decay,epsilon_min) \
&nbsp; b) Call get_experience() to obtain terminate & reward \
&nbsp; c) Render env \
&nbsp; d) IF terminate, append reward to episode_rewards & print mean reward of last 100 episodes \
&nbsp; e) IF we have attained minimum length for buffer, print 'model & target weights updated' 

Once required mean reward has been attained, print 'training complete', reset & close env

In [5]:
# for episode in tqdm(range(episodes)):
while True:    
    terminate = False
    while not terminate:
        epsilon = max(epsilon*epsilon_decay,epsilon_min)
        terminate, reward = agent.get_experience(actor, actor_target, critic, critic_target, epsilon, device)
        agent.env.render()
        if terminate:
            episode_rewards.append(reward)
            mean_reward = round(np.mean(episode_rewards[-100:]),3)
            print(f"mean reward: {mean_reward}")
            if len(buffer) == min_memory_len:
                print('model & target weights updated')
    if mean_reward > -6:
        print('training complete')
        break
        
env.reset()        
env.close() 

Score -38.96658240094558
mean reward: -38.967
Score -40.94329835069381
mean reward: -39.955
Score -41.255608473397224
mean reward: -40.388
Score -49.063982970467926
mean reward: -42.557
Score -43.004815843302254
mean reward: -42.647
Score -47.267952401844745
mean reward: -43.417
Score -43.367765819878045
mean reward: -43.41
Score -39.4344730452899
mean reward: -42.913
Score -38.722006072173166
mean reward: -42.447
Score -39.01721314659832
mean reward: -42.104
Score -46.0580254438627
mean reward: -42.464
Score -42.20276318372339
mean reward: -42.442
Score -52.32699440973057
mean reward: -43.202
Score -43.667407139906906
mean reward: -43.236
Score -37.13143326742037
mean reward: -42.829
Score -40.93063839194385
mean reward: -42.71
Score -44.675360791873665
mean reward: -42.826
Score -43.65017664329201
mean reward: -42.871
Score -39.025139993372306
mean reward: -42.669
Score -40.822530099564254
mean reward: -42.577
Score -51.20067311189741
mean reward: -42.987
Score -42.03935432062587
mea

Score -49.19599225216037
mean reward: -41.296
Score -40.65558161872294
mean reward: -41.311
Score -37.53130835798348
mean reward: -41.194
Score -45.533395722954715
mean reward: -41.16
Score -42.49736324616597
mean reward: -41.219
Score -39.89961927785977
mean reward: -41.158
Score -36.68075714018205
mean reward: -41.143
Score -41.69921728425272
mean reward: -41.165
Score -38.70222910413904
mean reward: -41.14
Score -34.4978495013589
mean reward: -41.09
Score -44.98688583175597
mean reward: -41.144
Score -48.01505631028271
mean reward: -41.215
Score -35.04359819601353
mean reward: -41.167
Score -38.23774435253906
mean reward: -41.167
Score -41.719775880862315
mean reward: -41.159
Score -46.916213780331795
mean reward: -41.21
Score -43.32855055004077
mean reward: -41.281
Score -38.70625572137947
mean reward: -41.269
Score -38.27957530533739
mean reward: -41.176
Score -42.93711404250696
mean reward: -41.189
Score -42.7444404785956
mean reward: -41.172
Score -38.14632342729657
mean reward:

Score -42.33225041034887
mean reward: -39.754
Score -39.30646516824249
mean reward: -39.743
Score -50.40344555170881
mean reward: -39.824
Score -36.01565591386011
mean reward: -39.799
Score -43.4427771947466
mean reward: -39.804
Score -40.16813686662595
mean reward: -39.759
Score -31.758271034604647
mean reward: -39.711
Score -32.192224529110916
mean reward: -39.621
Score -39.61018424030741
mean reward: -39.614
Score -38.25538040514176
mean reward: -39.606
Score -33.726130476961394
mean reward: -39.58
Score -41.14690631439518
mean reward: -39.603
Score -42.20871032854368
mean reward: -39.597
Score -41.15012855854732
mean reward: -39.618
Score -29.763147406664793
mean reward: -39.553
Score -44.57702199986803
mean reward: -39.594
Score -30.953703486994648
mean reward: -39.444
Score -42.14245383445147
mean reward: -39.385
Score -36.00504883164408
mean reward: -39.335
Score -40.05709901621753
mean reward: -39.305
Score -39.712227732449406
mean reward: -39.248
Score -45.62772870913254
mean 

Score -35.17352661337884
mean reward: -38.021
Score -40.02258861164556
mean reward: -37.936
Score -36.100739018231565
mean reward: -37.893
Score -34.70971295553368
mean reward: -37.844
Score -32.09549173310715
mean reward: -37.74
Score -33.78607701320145
mean reward: -37.694
Score -36.16521991631602
mean reward: -37.734
Score -35.57473612612683
mean reward: -37.705
Score -33.39922393408087
mean reward: -37.686
Score -39.762175050971486
mean reward: -37.744
Score -36.73438461488423
mean reward: -37.762
Score -37.48717533609892
mean reward: -37.755
Score -35.89979800529501
mean reward: -37.729
Score -38.26524179822266
mean reward: -37.778
Score -44.060116586640554
mean reward: -37.792
Score -37.063152710790916
mean reward: -37.718
Score -31.580775488971586
mean reward: -37.628
Score -42.35215082072381
mean reward: -37.682
Score -43.08764690227108
mean reward: -37.743
Score -42.36911735459932
mean reward: -37.825
Score -40.16006082556677
mean reward: -37.886
Score -38.32527842378033
mean 

Score -36.89736863694721
mean reward: -37.642
Score -36.43979393674748
mean reward: -37.679
Score -37.984152083063215
mean reward: -37.74
Score -46.75254077974085
mean reward: -37.911
Score -44.01581528575701
mean reward: -37.87
Score -28.351529113185027
mean reward: -37.82
Score -35.21482657775352
mean reward: -37.792
Score -39.6722184906726
mean reward: -37.77
Score -30.80262234317547
mean reward: -37.683
Score -43.28695971558177
mean reward: -37.784
Score -34.460497639612704
mean reward: -37.788
Score -39.15247723961584
mean reward: -37.805
Score -43.4649205995587
mean reward: -37.843
Score -40.06927016677559
mean reward: -37.889
Score -37.67467668856406
mean reward: -37.902
Score -34.03753736722644
mean reward: -37.871
Score -31.215660271170645
mean reward: -37.798
Score -36.05520901454495
mean reward: -37.723
Score -35.730617711582475
mean reward: -37.68
Score -33.31858604817714
mean reward: -37.661
Score -30.441716467400514
mean reward: -37.578
Score -29.98387056648311
mean rewar

...

Score -4.606210284317189
mean reward: -6.707
model & target weights updated
Score -6.410599201570475
mean reward: -6.686
model & target weights updated
Score -5.132152583304688
mean reward: -6.696
model & target weights updated
Score -8.194053044258384
mean reward: -6.751
model & target weights updated
Score -11.136191451268854
mean reward: -6.77
model & target weights updated
Score -8.446433971530169
mean reward: -6.763
model & target weights updated
Score -6.647474875784691
mean reward: -6.769
model & target weights updated
Score -8.069104491294341
mean reward: -6.742
model & target weights updated
Score -5.400602147415654
mean reward: -6.694
model & target weights updated
Score -4.031147351632692
mean reward: -6.683
model & target weights updated
Score -8.198676540431503
mean reward: -6.707
model & target weights updated
Score -7.268184290924189
mean reward: -6.665
model & target weights updated
Score -10.18479390867365
mean reward: -6.707
model & target weights updated
Score -4.479

Score -3.463636198673781
mean reward: -6.842
model & target weights updated
Score -3.099760123109133
mean reward: -6.833
model & target weights updated
Score -11.156216131924925
mean reward: -6.862
model & target weights updated
Score -4.387539692912568
mean reward: -6.833
model & target weights updated
Score -7.859414993927024
mean reward: -6.81
model & target weights updated
Score -7.822108540173671
mean reward: -6.844
model & target weights updated
Score -5.777588386473201
mean reward: -6.771
model & target weights updated
Score -3.1901836319358607
mean reward: -6.745
model & target weights updated
Score -3.2579379848198418
mean reward: -6.705
model & target weights updated
Score -2.339733730688102
mean reward: -6.667
model & target weights updated
Score -7.831299765759842
mean reward: -6.679
model & target weights updated
Score -3.341489621851865
mean reward: -6.612
model & target weights updated
Score -6.886585847360904
mean reward: -6.575
model & target weights updated
Score -3.5

Score -7.900177172480655
mean reward: -6.176
model & target weights updated
Score -3.6769963326771733
mean reward: -6.189
model & target weights updated
Score -10.055939966582816
mean reward: -6.211
model & target weights updated
Score -6.685557069890047
mean reward: -6.245
model & target weights updated
Score -6.474315565258274
mean reward: -6.241
model & target weights updated
Score -2.7079206768057227
mean reward: -6.232
model & target weights updated
Score -3.388707050525102
mean reward: -6.206
model & target weights updated
Score -7.509717729845144
mean reward: -6.23
model & target weights updated
Score -8.121193889129165
mean reward: -6.245
model & target weights updated
Score -4.8308270545257
mean reward: -6.224
model & target weights updated
Score -4.607316533640965
mean reward: -6.143
model & target weights updated
Score -5.605088249589707
mean reward: -6.127
model & target weights updated
Score -7.640843323246875
mean reward: -6.175
model & target weights updated
Score -9.749

SystemExit: 0

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#### Save model weights

Save model weights to "reacher_actor.pth" & "reacher_critic.pth" \
We can load trained weights for a new agent if we do not want to retrain

In [6]:
torch.save(actor.state_dict(), "reacher_actor.pth")
torch.save(critic.state_dict(), "reacher_critic.pth")

### Run this if you only want to use pre-trained weights & observe agent in action

#### Observe agent
Let's see how our agent performs for 10 episodes 

In [5]:
episodes = 10
env = gym.make('Reacher-v4')
buffer =  ExperienceReplay(max_memory_size)
trained_actor = Actor(num_states, hidden_size, num_actions).to(device)
trained_actor.load_state_dict(torch.load("reacher_actor.pth"))
trained_critic = Critic(num_states + num_actions, hidden_size, num_output).to(device)
trained_critic.load_state_dict(torch.load("reacher_critic.pth"))

agent = DDPG(env, buffer)
episode_rewards = []

for episode in tqdm(range(episodes)):
    terminate = False
    episode_reward = 0
    agent._reset()
    while not terminate:
        action = agent.select_optimal_action(trained_actor,device=device)
        next_state, reward, terminate, info = agent.env.step(action)
        env.render()
        episode_reward += reward
        agent.state = next_state
        if terminate:
            episode_rewards.append(episode_reward)
mean_reward = sum(episode_rewards)/len(episode_rewards)            
print("mean reward: {%.3f}" % mean_reward)

env.reset()        
env.close() 

100%|██████████| 10/10 [00:11<00:00,  1.19s/it]

mean reward: {-4.901}


SystemExit: 0

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#### Conclusion

As observed, our agent is able to reach the object most of the times within the time limit